# Modules and Global Variables

In [2]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, 
)

import torch, copy, json, re, os
from cleantext import clean
from tqdm import tqdm
from module.preprocess import preprocess
from module.preprocess import decorate_form, decorate_acd_pair, decorate_acd_pair_split, decorate_asc_pair, decorate_asc_pair_split

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)
    return j

def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

In [3]:
print(f'torch.__version__: {torch.__version__}')
print(f'torch.cuda.is_available(): {torch.cuda.is_available()}')
NGPU = torch.cuda.device_count()
print(f'NGPU: {NGPU}')

torch.__version__: 1.12.1
torch.cuda.is_available(): True
NGPU: 4


In [4]:
entity_property_pair = ['본품#품질', 
'제품 전체#일반', 
'본품#일반', 
'제품 전체#품질', 
'제품 전체#디자인', 
'본품#편의성', 
'제품 전체#편의성', 
'제품 전체#인지도', 
'패키지/구성품#디자인', 
'브랜드#일반', 
'제품 전체#가격']

# ### new
# entity_property_pair = [
#     '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
#     '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
#     '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
#     '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'
# ]

more_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

len(entity_property_pair)

11

In [34]:
ACD_CHECKPOINT = 'training_results/snunlp_kr_electra_discriminator_uncleaned_v4/acd/snunlp_kr_electra_discriminator_uncleaned_v4/checkpoint-3400'
ASC_CHECKPOINT = 'training_results/snunlp_kr_electra_discriminator_uncleaned_v4/asc/snunlp_kr_electra_discriminator_uncleaned_v4/checkpoint-360'

TEST_DATA_PATH = 'dataset/nikluge-sa-2022-test.jsonl'
EVAL_DATA_PATH = 'dataset/nikluge-sa-2022-dev.jsonl'

In [45]:
EVAL_MODE = True

if EVAL_MODE == True:
    TEST_DATA_PATH = EVAL_DATA_PATH
print('>>>>> >>>>> >>>>> ', TEST_DATA_PATH, ' <<<<< <<<<< <<<<<', '\n', sep='')

test_data = jsonlload(TEST_DATA_PATH)

if EVAL_MODE == True:
    for row in test_data:
        for annotation in row['annotation']:
            annotation.pop(1)
            
    true_data = copy.deepcopy(test_data)
    
    for row in test_data:
        row['annotation'] = []

    for idx, row in enumerate(true_data):
        print(row)
        if idx == 4:
            break
    print()
for idx, row in enumerate(test_data):
    print(row)
    if idx == 4:
        break

>>>>> >>>>> >>>>> dataset/nikluge-sa-2022-dev.jsonl <<<<< <<<<< <<<<<

{'id': 'nikluge-sa-2022-dev-00001', 'sentence_form': '깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면 금방 우러난다.', 'annotation': [['본품#편의성', 'positive']]}
{'id': 'nikluge-sa-2022-dev-00002', 'sentence_form': '목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약재는 약재이므로 용법은 알아서;;)신선한 한약풀 냄새가 욕실에 퍼져서 기분이 좋아졌다.', 'annotation': [['본품#일반', 'positive']]}
{'id': 'nikluge-sa-2022-dev-00003', 'sentence_form': '혹시 오래된 거 팔지 않나 고민했었는데 쑥향기 자체가 페퍼민트처럼 신선하고 포장도 깔끔하고 사용도 간편하고 참.. 우리나라 인터넷 시장도 좋은 거 같다.', 'annotation': [['본품#일반', 'positive'], ['패키지/구성품#디자인', 'positive']]}
{'id': 'nikluge-sa-2022-dev-00004', 'sentence_form': '전에는 목욕할 때 입욕제 넣고 하는 거 좋아하지 않았는데 한약재로 바꿔보니 약용까지는 몰라도 피로도 확 풀리고 기분도 좋아지고 돈 아깝다는 생각은 안 든다.', 'annotation': [['제품 전체#품질', 'positive']]}
{'id': 'nikluge-sa-2022-dev-00005', 'sentence_form': '나 또 시계 좋아하잖아.ㅋㅋ', 'annotation': [['제품 전체#일반', 'positive']]}

{'id': 'nikluge-sa-2022-dev-00001', 'sentence_form': '깔끔하게 부직포 포장으로 되어 있어서 그

# Load Model, Tokenizer, and Collator

In [46]:
acd_model = AutoModelForSequenceClassification.from_pretrained(ACD_CHECKPOINT)
acd_tokenizer = AutoTokenizer.from_pretrained(ACD_CHECKPOINT)

asc_model = AutoModelForSequenceClassification.from_pretrained(ASC_CHECKPOINT)
asc_tokenizer = AutoTokenizer.from_pretrained(ASC_CHECKPOINT)

In [47]:
decorate_form, decorate_acd_pair, decorate_acd_pair_split, decorate_asc_pair, decorate_asc_pair_split

(<function module.preprocess.decorate_form(form)>,
 <function module.preprocess.decorate_acd_pair(entity)>,
 <function module.preprocess.decorate_acd_pair_split(entity)>,
 <function module.preprocess.decorate_asc_pair(entity, sentiment)>,
 <function module.preprocess.decorate_asc_pair_split(entity, sentiment)>)

In [48]:
def predict_from_korean_form(acd_tokenizer, asc_tokenizer, acd_model, asc_model, data):

    acd_model.to(device)
    acd_model.eval()
    asc_model.to(device)
    asc_model.eval()

    for sentence in tqdm(data):
        form = sentence['sentence_form']
        # form = decorate_form(sentence['sentence_form'])
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is wrong: ", form)
            continue
        for pair in entity_property_pair:
            acd_pair = pair
            # acd_pair = decorate_acd_pair(pair)
            # acd_pair = decorate_acd_pair_split(pair)
            acd_encoded = acd_tokenizer(form, acd_pair, truncation=True, return_tensors="pt")
            acd_encoded = {k:v.to(device) for k,v in acd_encoded.items()}

            with torch.no_grad():
                acd_outputs = acd_model(**acd_encoded)
            
            ce_predictions = acd_outputs['logits'].argmax(-1)
            ce_result = tf_id_to_name[ce_predictions[0]]

            if ce_result == 'True':
                sentiments = ['positive', 'negative', 'neutral']
                asc_pairs = []
                for sentiment in sentiments:
                    asc_pair = '#'.join([pair, sentiment])
                    # asc_pair = decorate_asc_pair(pair, sentiment)
                    # asc_pair = decorate_asc_pair_split(pair, sentiment)
                    asc_pairs.append(asc_pair)

                positive = asc_tokenizer(form, asc_pairs[0], truncation=True, return_tensors="pt")
                positive = {k:v.to(device) for k,v in positive.items()}
                negative = asc_tokenizer(form, asc_pairs[1], truncation=True, return_tensors="pt")
                negative = {k:v.to(device) for k,v in negative.items()}
                neutral = asc_tokenizer(form, asc_pairs[2], truncation=True, return_tensors="pt")
                neutral = {k:v.to(device) for k,v in neutral.items()}

                with torch.no_grad():
                    positive_outputs = asc_model(**positive)
                    negative_outputs = asc_model(**negative)
                    neutral_outputs = asc_model(**neutral)

                pc_predictions = torch.tensor([positive_outputs['logits'][0][0], negative_outputs['logits'][0][0], neutral_outputs['logits'][0][0]]).argmax(-1)
                pc_result = polarity_id_to_name[pc_predictions]
                
                if pc_result == 'positive':
                    if pair == '패키지/구성품#가격':
                        print(f'{pair} found.')
                        pair = '패키지/ 구성품#가격'
                        print(f'corrected as {pair}')

                    sentence['annotation'].append([pair, pc_result])
                    # print(pair, pc_result)

    return data

In [49]:
pred_data = predict_from_korean_form(acd_tokenizer, asc_tokenizer, acd_model, asc_model, copy.deepcopy(test_data))

if EVAL_MODE == False:
    save_path = './'
    file_name = 'snunlp_kr_electra_discriminator_uncleaned_v4.json'

    jsondump(pred_data, os.path.join(save_path, file_name))
    pred_data = jsonload(os.path.join(save_path, file_name))
    
len(test_data), len(pred_data)

100%|██████████| 2794/2794 [08:05<00:00,  5.75it/s]


In [52]:
for idx, row in enumerate(pred_data):
    print(row)
    if idx == 4:
        break

{'id': 'nikluge-sa-2022-dev-00001', 'sentence_form': '깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면 금방 우러난다.', 'annotation': [['제품 전체#품질', 'positive']]}
{'id': 'nikluge-sa-2022-dev-00002', 'sentence_form': '목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약재는 약재이므로 용법은 알아서;;)신선한 한약풀 냄새가 욕실에 퍼져서 기분이 좋아졌다.', 'annotation': [['본품#품질', 'positive']]}
{'id': 'nikluge-sa-2022-dev-00003', 'sentence_form': '혹시 오래된 거 팔지 않나 고민했었는데 쑥향기 자체가 페퍼민트처럼 신선하고 포장도 깔끔하고 사용도 간편하고 참.. 우리나라 인터넷 시장도 좋은 거 같다.', 'annotation': [['본품#편의성', 'positive']]}
{'id': 'nikluge-sa-2022-dev-00004', 'sentence_form': '전에는 목욕할 때 입욕제 넣고 하는 거 좋아하지 않았는데 한약재로 바꿔보니 약용까지는 몰라도 피로도 확 풀리고 기분도 좋아지고 돈 아깝다는 생각은 안 든다.', 'annotation': [['제품 전체#품질', 'positive']]}
{'id': 'nikluge-sa-2022-dev-00005', 'sentence_form': '나 또 시계 좋아하잖아.ㅋㅋ', 'annotation': [['제품 전체#일반', 'positive']]}


# Scoring

In [55]:
def evaluation_f1(true_data, pred_data):

    true_data_list = true_data
    pred_data_list = pred_data

    ce_eval = {
        'TP': 0,
        'FP': 0,
        'FN': 0,
        'TN': 0
    }

    pipeline_eval = {
        'TP': 0,
        'FP': 0,
        'FN': 0,
        'TN': 0
    }

    for i in range(len(true_data_list)):

        # TP, FN checking
        is_ce_found = False
        is_pipeline_found = False
        for y_ano  in true_data_list[i]['annotation']:
            y_category = y_ano[0]
            y_polarity = y_ano[1]

            for p_ano in pred_data_list[i]['annotation']:
                p_category = p_ano[0]
                p_polarity = p_ano[1]

                if y_category == p_category:
                    is_ce_found = True
                    if y_polarity == p_polarity:
                        is_pipeline_found = True

                    break

            if is_ce_found is True:
                ce_eval['TP'] += 1
            else:
                ce_eval['FN'] += 1

            if is_pipeline_found is True:
                pipeline_eval['TP'] += 1
            else:
                pipeline_eval['FN'] += 1

            is_ce_found = False
            is_pipeline_found = False

        # FP checking
        for p_ano in pred_data_list[i]['annotation']:
            p_category = p_ano[0]
            p_polarity = p_ano[1]

            for y_ano  in true_data_list[i]['annotation']:
                y_category = y_ano[0]
                y_polarity = y_ano[1]

                if y_category == p_category:
                    is_ce_found = True
                    if y_polarity == p_polarity:
                        is_pipeline_found = True

                    break

            if is_ce_found is False:
                ce_eval['FP'] += 1

            if is_pipeline_found is False:
                pipeline_eval['FP'] += 1

    ce_precision = ce_eval['TP']/(ce_eval['TP']+ce_eval['FP'])
    ce_recall = ce_eval['TP']/(ce_eval['TP']+ce_eval['FN'])

    ce_result = {
        'Precision': ce_precision,
        'Recall': ce_recall,
        'F1': 2*ce_recall*ce_precision/(ce_recall+ce_precision)
    }

    pipeline_precision = pipeline_eval['TP']/(pipeline_eval['TP']+pipeline_eval['FP'])
    pipeline_recall = pipeline_eval['TP']/(pipeline_eval['TP']+pipeline_eval['FN'])

    pipeline_result = {
        'Precision': pipeline_precision,
        'Recall': pipeline_recall,
        'F1': 2*pipeline_recall*pipeline_precision/(pipeline_recall+pipeline_precision)
    }

    return {
        'category extraction result': ce_result,
        'entire pipeline result': pipeline_result
    }

In [56]:
evaluation_f1(true_data, pred_data)

{'category extraction result': {'Precision': 0.7258467023172905,
  'Recall': 0.661253653783696,
  'F1': 0.6920462270564242},
 'entire pipeline result': {'Precision': 0.7122994652406417,
  'Recall': 0.6489119844105229,
  'F1': 0.6791298436437797}}

# Inference Test

In [12]:
# form = '패키지에 보니 허브한방추출물과 옷나무 껍질 추출물이 들어갔다고 해서 한방향이 날줄알았는데 제품제형은 투명하고 향은 상큼한 향이랄까요?'
# form = '최근 북한 미사일 발사 등 도발 및 한미·한미일 대응'
# pair = '패키지/구성품#일반'

In [13]:
# # tokenized_data = acd_tokenizer(form, pair, padding='max_length', max_length=256, truncation=True)
# acd_encoded = acd_tokenizer(form, pair, truncation=True, return_tensors="pt")
# acd_encoded = {k:v.to(device) for k,v in acd_encoded.items()}

# with torch.no_grad():
#     acd_outputs = acd_model(**acd_encoded)

# ce_predictions = acd_outputs['logits'].argmax(-1)
# ce_result = tf_id_to_name[ce_predictions[0]]
# print(acd_outputs['logits'])
# print(ce_predictions)
# print(ce_result)

In [14]:
# asc_encoded = asc_tokenizer(form, pair, truncation=True, return_tensors="pt")
# asc_encoded = {k:v.to(device) for k,v in asc_encoded.items()}

# with torch.no_grad():
#     asc_outputs = asc_model(**asc_encoded)

# pc_predictions = asc_outputs['logits'].argmax(-1)
# pc_result = polarity_id_to_name[pc_predictions[0]]
# print(asc_outputs['logits'])
# print(pc_predictions)
# print(pc_result)